In [1]:
import numpy as np
import pandas as pd
import gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task


'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/zhenghao/.conda/envs/autogluon/lib/python3.10/site-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")


In [2]:
train = pd.read_csv('/home/zhenghao/kaggle_grealish/grealish_files/kaggle_1/lightautoml/train.csv')

In [3]:
print(train.columns)


Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')


In [4]:
print(X_train.columns)
print(X_val.columns)


NameError: name 'X_train' is not defined

In [5]:
X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train['class'])

In [6]:
task = Task('binary')

In [7]:
automl = TabularAutoML(
    task=task,  # 指定任务类型
    timeout=600 * 3600,  # 设定训练时间上限为600小时
    cpu_limit=24,  # 使用的CPU核心数量限制为12
    gpu_ids='0',  # 使用第一个GPU
    general_params={"use_algos": [['linear_l2', 'lgb', 'lgb_tuned', 'cb']]},  # 让系统自动选择合适的算法
    reader_params={'n_jobs': 12, 'cv': 5, 'random_state': 42, 'advanced_roles': True}  # 数据读取参数
)

out_of_fold_predictions = automl.fit_predict(
    X_train, valid_data=X_val,  # 训练集和验证集
    roles={
        'target': 'class',  # 目标列
        'drop': ['id'],  # 丢弃的列
    }, 
    verbose=4  # 显示详细的训练信息
)

[10:29:22] Stdout logging level is DEBUG.


[10:29:22] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[10:29:22] Task: binary

[10:29:22] Start automl preset with listed constraints:
[10:29:22] - time: 2160000.00 seconds
[10:29:22] - CPU: 24 cores
[10:29:22] - memory: 16 GB

[10:29:22] Train data shape: (2493556, 22)

[10:29:34] Feats was rejected during automatic roles guess: []
[10:29:35] Layer 1 train process start. Time left 2159986.75 secs
[10:31:34] Linear model: C = 1e-05 score = 0.9010560349808924
[10:32:05] Linear model: C = 5e-05 score = 0.9271357831713997
[10:32:38] Linear model: C = 0.0001 score = 0.934132529371531
[10:33:34] Linear model: C = 0.0005 score = 0.9443477168199919
[10:34:31] Linear model: C = 0.001 score = 0.9468095333456115
[10:35:47] Linear model: C = 0.005 score = 0.9498714877568295
[10:35:48] Linear model: C = 0.01 score = 0.9498714877568295
[10:35:49] Linear model: C = 0.05 score = 0.9498714877568295
[10:35:49] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting a

In [8]:
del train, X_train, X_val
gc.collect()

955

In [9]:
test = pd.read_csv('/home/zhenghao/kaggle_grealish/grealish_files/kaggle_1/lightautoml/test.csv')
ss = pd.read_csv('/home/zhenghao/kaggle_grealish/grealish_files/kaggle_1/lightautoml/sample_submission.csv')

In [14]:
# Use the automl model to predict probabilities
pred = automl.predict(test).data[:, 0]

# Convert probabilities to 'p' or 'e' based on the threshold of 0.5
pred_labels = np.where(pred > 0.5, 'e', 'p')

# Assign the modified labels to the appropriate column in your DataFrame
ss['class'] = pred_labels  # Make sure the column name is correctly specified

# Save the DataFrame to CSV, ensuring you do not include the index
ss.to_csv('CHONGCI2.csv', index=False)

# Optional: Print the first few rows to verify the contents before saving
print(ss.head())


        id class
0  3116945     e
1  3116946     p
2  3116947     p
3  3116948     p
4  3116949     e
